In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import time

def get_weather_today():
    url = "https://www.accuweather.com/fr/tn/tunis/321398/hourly-weather-forecast/321398"
    time.sleep(5)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("Erreur lors de la récupération des données pour Tunis")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Lists to store the data
    dates = []
    temperatures = []
    
    # Get today's date
    today = datetime.now().date()
    
    # Find all date and temperature elements
    date_elements = soup.find_all('h2', class_='date')
    temp_elements = soup.find_all('div', class_='temp metric')

    # Extract data and store in lists
    for date_elem, temp_elem in zip(date_elements, temp_elements):
        # Extract time part from the date element (e.g., "19 h")
        time_str = date_elem.get_text(strip=True).split(' ')[0]  # Only keep the hour part
        hour = int(time_str)
        
        # Combine today's date with the hour to form a datetime object
        date_time = datetime.combine(today, datetime.min.time()) + timedelta(hours=hour)
        
        dates.append(date_time)
        temperatures.append(temp_elem.get_text(strip=True))

    # Format data as a list of dictionaries
    weather_data = [{'date': date, 'température': temp} for date, temp in zip(dates, temperatures)]
    return weather_data



In [3]:
# Fetch weather data
weather_data = get_weather_today()

# Convert to DataFrame
df_today = pd.DataFrame(weather_data)
print(df_today)


                 date température
0 2024-11-04 21:00:00         21°
1 2024-11-04 22:00:00         20°
2 2024-11-04 23:00:00         20°


In [5]:
import plotly.express as px
fig = px.line(df_today, x='date', y='température', title="Température Heure par Heure",
              labels={'date': 'Date et Heure', 'température': 'Température (°C)'})

# Affichage du graphique
fig.show()